In [1]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
from sklearn.model_selection import train_test_split

Mounted at /content/drive


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
'''def get_data():

#----1. Prepare the Training Data
#----Features: [Weight in grams, Texture scale 1-10]
#----Labels: 0 for Apple, 1 for Orange
    features_train = [
        [140, 8],  #----Apple 1
        [150, 7],  #----Apple 2
        [130, 9],  #----Apple 3
        [200, 2],  #----Orange 1
        [220, 3],  #----Orange 2
        [210, 1],  #----Orange 3
    ]
    classes_train = [0, 0, 0, 1, 1, 1]

    features_test = [
        [170, 8],  #----Apple 4
        [190, 4]   #----Orange 4
    ]
    classes_test = [0, 1]

#----Give meaningful names to features and classes. Classes must be in order of their labels.
    feature_names = ["weight in grams","texture"]
    class_names = ["apple","orange"]

    return features_train,classes_train,features_test,classes_test,feature_names,class_names'''

In [ ]:
'''features_train,classes_train,features_test,classes_test,feature_names,class_names = get_data()'''
feature_names,class_names,features_train,classes_train,features_test,classes_test


(['weight in grams', 'texture'],
 ['apple', 'orange'],
 [[140, 8], [150, 7], [130, 9], [200, 2], [220, 3], [210, 1]],
 [0, 0, 0, 1, 1, 1],
 [[170, 8], [190, 4]],
 [0, 1])

In [3]:
#----The last column is the class
def get_data_from_csv(file_name,class_names):

    data = np.genfromtxt(file_name, delimiter=',', names=True, filling_values=0, dtype=None, \
ndmin=1)
    # print(f"Extracting from {file_name} and got {data}")
#----Extract all except the last column name as feature names
    feature_names = list(data.dtype.names[:-1])
#----Extract unique values from last column as class names
    # print(f"All classes names {data[data.dtype.names[-1]]}")
    if not class_names:
        class_names = np.unique(data[data.dtype.names[-1]]).tolist()
        # print(f"Using new class names {class_names}")
    # else:
        # print(f"Using previous class names {class_names}")
    # print(f"Feature names {feature_names} Class names {class_names}")

#----Extract all except the last column as features
    features = data[feature_names].tolist()
    # print(f"Features are {features}")
#----Extract last column as classes, and convert to integers
    all_classes = list(data[data.dtype.names[-1]])
    # print(f"All classes {all_classes}")
    classes = np.zeros(len(all_classes), dtype=int)
    for index, value in enumerate(all_classes):
        classes[index] = class_names.index(value)
    classes = classes.tolist()
    # print(f"All integer classes {classes}")

    return features,classes,feature_names,class_names

In [4]:
features_train,classes_train,feature_names,class_names = get_data_from_csv("/content/drive/MyDrive/Colab Notebooks/KNN-fruit_train.csv",None);
features_test,classes_test,feature_names,class_names = get_data_from_csv("/content/drive/MyDrive/Colab Notebooks/KNN-fruit_test.csv",class_names);
feature_names,class_names,features_train,classes_train,features_test,classes_test

(['weight_in_grams', 'texture'],
 ['Apple', 'Orange'],
 [(140, 8), (150, 7), (130, 9), (200, 2), (220, 3), (210, 1)],
 [0, 0, 0, 1, 1, 1],
 [(190, 4), (170, 8)],
 [1, 0])

In [5]:
def get_predictions(K,features_train,classes_train,features_test):

#----Initialize the K-NN Classifier
    knn = KNeighborsClassifier(n_neighbors=K)
#----Train the model
    knn.fit(features_train, classes_train)
#----Predict for the test data
    predictions = knn.predict(features_test)

    return knn,predictions

In [6]:
knn,predictions = get_predictions(3,features_train,classes_train,features_test)
knn,predictions

(KNeighborsClassifier(n_neighbors=3), array([1, 0]))

In [7]:
def print_predictions(knn,predictions,features_test,classes_test,class_names):

#----Check the probability (How sure is the model?)
    probability = knn.predict_proba(features_test)
#----Output the result for each test data
    for index, value in enumerate(classes_test):
        print(f"The {class_names[value]} is classified as: {class_names[predictions[index]]}")
        print(f"Confidence is {probability[index][predictions[index]] * 100:.0f}%")

#----Compute the accuracy
    accuracy = accuracy_score(classes_test,predictions)
    print(f"The accuracy is {accuracy}")

In [8]:
print_predictions(knn,predictions,features_test,classes_test,class_names)

The Orange is classified as: Orange
Confidence is 100%
The Apple is classified as: Apple
Confidence is 67%
The accuracy is 1.0


In [11]:
# Load the car data
car_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/preprocessed_data.csv")
print(car_data.head())

   Style  Volume_Normalized  Doors_Normalized
0  Sedan           2.785425                 7
1    SUV           3.477733                10
2  Sedan           3.186235                 7
3    SUV           3.951417                10
4    SUV           3.951417                10


In [16]:
# Separate features and target
X = car_data.iloc[:, 1:]   # Columns 1 and 2 (Volume_Normalized, Doors_Normalized)
y = car_data.iloc[:, 0]     # Column 0 (Style) - it's FIRST not last!

# Split 80% train, 20% test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
print(f"Training: {len(X_train)}, Testing: {len(X_test)}")

Training: 120, Testing: 30


In [17]:
# Create and save Training.csv
train_data = pd.concat([X_train.reset_index(drop=True),
                        y_train.reset_index(drop=True)], axis=1)
train_data.to_csv('Training.csv', index=False)

# Create and save Testing.csv
test_data = pd.concat([X_test.reset_index(drop=True),
                       y_test.reset_index(drop=True)], axis=1)
test_data.to_csv('Testing.csv', index=False)

In [18]:
# Test K from 1 to 20
accuracy_results = []
best_accuracy = 0
best_k = 1
best_knn = None

for k in range(1, 21):
    knn, predictions = get_predictions(k, X_train.values, y_train.values, X_test.values)
    accuracy = accuracy_score(y_test, predictions)
    accuracy_results.append({'K': k, 'Accuracy': accuracy})

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_k = k
        best_knn = knn

    print(f"K={k}: Accuracy={accuracy:.4f}")

print(f"\nBest K: {best_k} with accuracy: {best_accuracy:.4f}")

K=1: Accuracy=0.6000
K=2: Accuracy=0.3667
K=3: Accuracy=0.5667
K=4: Accuracy=0.5667
K=5: Accuracy=0.4667
K=6: Accuracy=0.5333
K=7: Accuracy=0.5667
K=8: Accuracy=0.6000
K=9: Accuracy=0.5667
K=10: Accuracy=0.6000
K=11: Accuracy=0.5667
K=12: Accuracy=0.6000
K=13: Accuracy=0.5667
K=14: Accuracy=0.5667
K=15: Accuracy=0.5667
K=16: Accuracy=0.6000
K=17: Accuracy=0.6000
K=18: Accuracy=0.5667
K=19: Accuracy=0.5667
K=20: Accuracy=0.6333

Best K: 20 with accuracy: 0.6333


In [19]:
# Save accuracy results
accuracy_df = pd.DataFrame(accuracy_results)
accuracy_df.to_csv('Accuracy.csv', index=False)
print(accuracy_df)

     K  Accuracy
0    1  0.600000
1    2  0.366667
2    3  0.566667
3    4  0.566667
4    5  0.466667
5    6  0.533333
6    7  0.566667
7    8  0.600000
8    9  0.566667
9   10  0.600000
10  11  0.566667
11  12  0.600000
12  13  0.566667
13  14  0.566667
14  15  0.566667
15  16  0.600000
16  17  0.600000
17  18  0.566667
18  19  0.566667
19  20  0.633333


In [20]:
# Get predictions and confidence using best K
final_predictions = best_knn.predict(X_test.values)
probabilities = best_knn.predict_proba(X_test.values)
confidence = np.max(probabilities, axis=1)

# Add columns to test data
test_data['Prediction'] = final_predictions
test_data['Confidence'] = confidence

# Save updated Testing.csv
test_data.to_csv('Testing.csv', index=False)
print(test_data.head(10))

   Volume_Normalized  Doors_Normalized   Style Prediction  Confidence
0           4.315789                10     SUV        SUV        0.85
1           3.259109                 7     SUV      Sedan        0.70
2           2.858300                 4    Jeep      Sedan        0.55
3           2.712551                 7  Pickup      Sedan        0.75
4           3.368421                 7   Sedan      Sedan        0.60
5           3.259109                 7     SUV      Sedan        0.70
6           2.566802                 7     SUV      Sedan        0.70
7           4.024291                 7     SUV        SUV        0.50
8           2.821862                 7     SUV      Sedan        0.65
9           3.987854                10    Jeep        SUV        0.85


In [21]:
from google.colab import files
files.download('Training.csv')
files.download('Testing.csv')
files.download('Accuracy.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>